In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Input, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split

import tensorflow as tf
from load_policy import load_policy
import pickle
import numpy as np
import gym

import tf_util


Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [3]:
expert_policy_file = "experts/Ant-v1.pkl"
env_name = "Ant-v1"
rounds = 1000
max_steps = 1000

In [4]:
data = pickle.load(open(expert_policy_file, "rb"))
policy_fn = load_policy(expert_policy_file)

obs (1, 111) (1, 111)


In [6]:
observations = []
actions = []
rewards = []

with tf.Session() as sess:
#     tf_util.initialize()
    env = gym.make(env_name)
    for i in range(rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r_, done, _ = env.step(action)
            rewards.append(r_)
            r += r_
            if done:
                break
        if i and i%100 == 0:
            print(i)
            

[2018-06-06 11:31:00,324] Making new env: Ant-v1


100
200
300
400
500
600
700
800
900


In [7]:
observations = np.array(observations)
actions = np.vstack(actions)
rewards = np.array(rewards)

obs_train, obs_valid, act_train, act_valid = \
    train_test_split(observations, actions, test_size = 0.2, random_state = 1998)

In [8]:
actions.shape

(992759, 8)

In [15]:
input_dim = observations.shape[-1]
bc = Sequential()
bc.add(Dense(512, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
bc.add(Dense(1024, input_dim=input_dim, kernel_initializer='normal', activation='relu'))

bc.add(Dense(512, kernel_initializer='normal', activation='relu'))
# bc.add(BatchNormalization())
# bc.add(Dropout(0.5))
bc.add(Dense(64, kernel_initializer='normal', activation='relu'))

bc.add(Dense(8, kernel_initializer='normal'))


In [16]:
bc.compile(optimizer='adam',loss='mse', metrics=['accuracy', 'mae'])

In [17]:
bc.fit(obs_train, act_train, validation_data=[obs_valid, act_valid],  epochs=20, batch_size= 512)

Train on 794207 samples, validate on 198552 samples
Epoch 1/20
794207/794207 [==============================] - 6s 7us/step - loss: 0.0026 - acc: 0.9041 - mean_absolute_error: 0.0305 - val_loss: 0.0014 - val_acc: 0.9121 - val_mean_absolute_error: 0.0262
Epoch 2/20
794207/794207 [==============================] - 6s 7us/step - loss: 5.1475e-04 - acc: 0.9519 - mean_absolute_error: 0.0154 - val_loss: 4.5955e-04 - val_acc: 0.9560 - val_mean_absolute_error: 0.0149
Epoch 3/20
794207/794207 [==============================] - 6s 7us/step - loss: 3.5147e-04 - acc: 0.9601 - mean_absolute_error: 0.0129 - val_loss: 4.7685e-04 - val_acc: 0.9523 - val_mean_absolute_error: 0.0146
Epoch 4/20
794207/794207 [==============================] - 6s 7us/step - loss: 2.7700e-04 - acc: 0.9647 - mean_absolute_error: 0.0114 - val_loss: 4.4162e-04 - val_acc: 0.9538 - val_mean_absolute_error: 0.0143
Epoch 5/20
794207/794207 [==============================] - 6s 7us/step - loss: 2.3862e-04 - acc: 0.9675 - mean_abso

In [18]:
rewards.sum()/1000

4774.063403404581

In [19]:
o = np.random.random(111)[None, :]
bc.predict(o)

array([[ 0.09217432, -0.3808537 ,  0.94491684, -0.20431328, -0.83760446,
         0.4058509 , -0.24325673,  0.25300175]], dtype=float32)

In [20]:
with tf.Session():
    print(policy_fn(o))

[[ 0.05754467 -0.6614223  -0.08244892 -0.2566884   0.15998185 -0.3632475
   0.22522736  0.37842995]]


In [21]:
m_rewards = []

def run_model(m, m_rounds, max_steps):
    env = gym.make(env_name)
    for i in range(m_rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = m.predict(obs[None,:])
#             observations.append(obs)
#             actions.append(action)
            obs, r_, done, _ = env.step(action)
            m_rewards.append(r_)
#             r += r_
            if done:
                break

In [24]:
mr = 1000
mstep = 1000
run_model(bc, mr, mstep)

[2018-06-06 11:57:11,665] Making new env: Ant-v1


In [25]:
sum(m_rewards)/mr

4812.432086982931

In [28]:
len(m_rewards)

1003534